In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from scipy import sparse
from sklearn.feature_extraction import FeatureHasher
from scipy.sparse import coo_matrix,csr_matrix,csc_matrix, hstack
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn import linear_model
import gc
from sklearn import preprocessing

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
predictors_target_11 = ['ruta_freq', 'clien_freq', 'agen_freq', 'prod_freq',
       'agen_for_log_de', 'ruta_for_log_de', 'cliente_for_log_de',
       'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_agen_for_log_sum', 'corr', 't_min_6', 't_min_2',
       't_min_3', 't_min_4', 't_min_5', 't2_min_t6', 't3_min_t6',
       't4_min_t6', 't5_min_t6', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod', 'LR_prod_corr',
       'target', 't_m_6_cum', 't_m_5_cum', 't_m_4_cum', 't_m_3_cum',
       't_m_2_cum', 'NombreCliente', 'weight', 'weight_per_piece', 'pieces']

In [4]:
predictors_11 = ['ruta_freq', 'clien_freq', 'agen_freq',
       'prod_freq', 'agen_for_log_de', 'ruta_for_log_de',
       'cliente_for_log_de', 'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_agen_for_log_sum', 'corr', 't_min_6', 't_min_2',
       't_min_3', 't_min_4', 't_min_5', 't2_min_t6', 't3_min_t6',
       't4_min_t6', 't5_min_t6', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod', 'LR_prod_corr',
       't_m_6_cum', 't_m_5_cum', 't_m_4_cum', 't_m_3_cum', 't_m_2_cum',
       'NombreCliente', 'weight', 'weight_per_piece', 'pieces']

In [5]:
train_pivot_xgb_time2 = pd.read_csv('train_pivot_34567_to_9.csv',usecols = predictors_target_11,dtype = np.float32)

In [6]:
train_pivot_xgb_time2.columns.values

array(['ruta_freq', 'clien_freq', 'agen_freq', 'prod_freq',
       'agen_for_log_de', 'ruta_for_log_de', 'cliente_for_log_de',
       'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_agen_for_log_sum', 'corr', 't_min_6', 't_min_2',
       't_min_3', 't_min_4', 't_min_5', 't2_min_t6', 't3_min_t6',
       't4_min_t6', 't5_min_t6', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod', 'LR_prod_corr',
       'target', 't_m_6_cum', 't_m_5_cum', 't_m_4_cum', 't_m_3_cum',
       't_m_2_cum', 'NombreCliente', 'weight', 'weight_per_piece', 'pieces'], dtype=object)

In [7]:
train_pivot_xgb_time2.head()

,ruta_freq,clien_freq,agen_freq,prod_freq,agen_for_log_de,ruta_for_log_de,cliente_for_log_de,producto_for_log_de,agen_ruta_for_log_de,agen_cliente_for_log_de,...,target,t_m_6_cum,t_m_5_cum,t_m_4_cum,t_m_3_cum,t_m_2_cum,NombreCliente,weight,weight_per_piece,pieces
0,11.0,160.0,4017.0,2159.0,3.936934,3.095841,2.897803,3.106304,3.095841,3.015667,...,4.574711,0.000000,0.000000,0.000000,0.000000,0.000000,233.0,600.0,75.00,8.0
1,548.0,160.0,12574.0,15736.0,3.489749,3.208231,2.897803,1.563950,2.877003,2.877003,...,2.639057,3.044523,5.877736,8.650325,11.422914,14.367352,18989.0,640.0,NaN,NaN
2,548.0,160.0,12574.0,7432.0,3.489749,3.208231,2.897803,2.637196,2.877003,2.877003,...,2.397895,1.386294,1.386294,1.386294,1.386294,1.386294,18989.0,180.0,30.00,6.0
3,548.0,160.0,12574.0,17261.0,3.489749,3.208231,2.897803,2.674463,2.877003,2.877003,...,3.784190,3.784190,7.218177,11.024839,14.580188,18.191105,18989.0,450.0,56.25,8.0
4,548.0,160.0,12574.0,87803.0,3.489749,3.208231,2.897803,1.730373,2.877003,2.877003,...,4.682131,4.795791,9.202510,12.963710,17.212206,21.786917,18989.0,340.0,42.50,8.0


In [8]:
train_pivot_xgb_time2.shape

(10385350, 44)

In [9]:
train_pivot_56789_to_11 = pd.read_csv('train_pivot_56789_to_11_private.csv',dtype=np.float32)

In [10]:
train_pivot_56789_to_11.head()

,id,ruta_freq,clien_freq,agen_freq,prod_freq,agen_for_log_de,ruta_for_log_de,cliente_for_log_de,producto_for_log_de,agen_ruta_for_log_de,...,LR_prod_corr,t_m_6_cum,t_m_5_cum,t_m_4_cum,t_m_3_cum,t_m_2_cum,NombreCliente,weight,weight_per_piece,pieces
0,1547831.0,713.0,166.0,12208.0,91578.0,3.523074,3.222417,2.890955,1.724493,2.835826,...,7.372772,3.761200,8.009695,12.584406,17.084215,21.766348,5954.0,340.0,42.500,8.0
1,6825659.0,713.0,166.0,12208.0,10780.0,3.523074,3.222417,2.890955,3.987424,2.835826,...,18.803841,0.000000,2.833213,6.165418,8.730368,11.949243,5954.0,480.0,NaN,NaN
2,5853787.0,713.0,166.0,12208.0,11047.0,3.523074,3.222417,2.890955,2.694399,2.835826,...,11.708699,1.791759,4.564348,6.962244,10.006766,13.002499,5954.0,567.0,NaN,NaN
3,2316053.0,713.0,166.0,12208.0,4912.0,3.523074,3.222417,2.890955,2.392465,2.835826,...,10.535754,0.000000,0.000000,0.000000,0.000000,1.386294,5954.0,435.0,54.375,8.0
4,900676.0,713.0,166.0,12208.0,10614.0,3.523074,3.222417,2.890955,2.745177,2.835826,...,12.040730,1.098612,1.098612,1.098612,2.890372,5.934894,5954.0,248.0,62.000,4.0


In [18]:
train_pivot_56789_to_11.columns.values

array(['id', 'ruta_freq', 'clien_freq', 'agen_freq', 'prod_freq',
       'agen_for_log_de', 'ruta_for_log_de', 'cliente_for_log_de',
       'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_agen_for_log_sum', 'corr', 't_min_6', 't_min_2',
       't_min_3', 't_min_4', 't_min_5', 't2_min_t6', 't3_min_t6',
       't4_min_t6', 't5_min_t6', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod', 'LR_prod_corr',
       't_m_6_cum', 't_m_5_cum', 't_m_4_cum', 't_m_3_cum', 't_m_2_cum',
       'NombreCliente', 'weight', 'weight_per_piece', 'pieces'], dtype=object)

### begin xgboost
---------------

In [11]:
train_feature_11 = train_pivot_xgb_time2.drop(['target'],axis = 1)
train_label_11 = train_pivot_xgb_time2['target']

dtrain_11 = xgb.DMatrix(train_feature_11,label = train_label_11,missing= np.nan)

In [92]:
#right now the best is this parameter set with 700 round
param_11 = {'booster':'gbtree',
         'nthread': 8,
         'max_depth':8, 
         'eta':0.015,
         'silent':1,
         'subsample':0.5, 
         'objective':'reg:linear',
         'eval_metric':'rmse',
         'colsample_bytree':0.5}

In [43]:
# # for cv
# num_round = 1400

# cvresult = xgb.cv(param_11, dtrain_11, num_round, nfold=4,show_stdv=False,
#                         seed = 0, verbose_eval = 1)
# print(cvresult.tail())

In [131]:
# for real train
num_round = 2200
evallist = [(dtrain_11,'train')]
bst = xgb.train(param_11, dtrain_11, num_round,evallist)

[0]	train-rmse:1.35068
[1]	train-rmse:1.33297
[2]	train-rmse:1.31555
[3]	train-rmse:1.29857
[4]	train-rmse:1.28177
[5]	train-rmse:1.26544
[6]	train-rmse:1.2493
[7]	train-rmse:1.23334
[8]	train-rmse:1.21778
[9]	train-rmse:1.20234
[10]	train-rmse:1.18717
[11]	train-rmse:1.17227
[12]	train-rmse:1.15764
[13]	train-rmse:1.14325
[14]	train-rmse:1.12907
[15]	train-rmse:1.11514
[16]	train-rmse:1.10143
[17]	train-rmse:1.08799
[18]	train-rmse:1.07478
[19]	train-rmse:1.06182
[20]	train-rmse:1.0491
[21]	train-rmse:1.03677
[22]	train-rmse:1.02445
[23]	train-rmse:1.01235
[24]	train-rmse:1.00063
[25]	train-rmse:0.989205
[26]	train-rmse:0.977832
[27]	train-rmse:0.966595
[28]	train-rmse:0.955548
[29]	train-rmse:0.944862
[30]	train-rmse:0.934253
[31]	train-rmse:0.923844
[32]	train-rmse:0.913594
[33]	train-rmse:0.903533
[34]	train-rmse:0.893702
[35]	train-rmse:0.884046
[36]	train-rmse:0.874552
[37]	train-rmse:0.865223
[38]	train-rmse:0.856063
[39]	train-rmse:0.847138
[40]	train-rmse:0.838336
[41]	train-r

- 1600 : 0.442933
- 1800 : 0.442292
- 2200 : 0.441234

In [141]:
bst.save_model('bst11_2200_eta0015.model')

In [56]:
dtest_11 = xgb.DMatrix(train_pivot_56789_to_11.drop(['id'],axis =1), missing=np.nan)

In [132]:
submission_11_all_train = pd.DataFrame()
submission_11_all_train = train_pivot_56789_to_11[['id']].copy()
submission_11_all_train['predict'] = bst.predict(dtest_11)
submission_11_all_train.reset_index(drop = True,inplace = True)

In [96]:
submission_11_all_train['predict'].describe()

count    3.460866e+06
mean     1.561886e+00
std      7.065410e-01
min     -3.176572e-01
25%      1.056087e+00
50%      1.396147e+00
75%      1.880355e+00
max      7.308497e+00
Name: predict, dtype: float64

In [97]:
submission_11_all_train.head()

,id,predict
0,1547831.0,4.339578
1,6825659.0,2.999748
2,5853787.0,2.638007
3,2316053.0,1.133329
4,900676.0,2.027621


In [133]:
submission_11_all_train['id'] = submission_11_all_train['id'].astype(int)

### concat result

In [29]:
submission_10 = pd.read_csv('week10_private.csv',index_col = 0)

In [38]:
submission_10.head()

,id,predict
0,1569352,2.389577
1,6667200,3.532464
2,1592616,2.962314
3,3909690,4.135859
4,3659672,3.563721


In [33]:
submission_all_train = pd.concat([submission_10,submission_11_all_train],axis =0)
submission_all_train['predict'] = submission_all_train['predict'].apply(np.expm1)
submission_all_train.rename(columns = {'predict':'Demanda_uni_equil'},inplace = True)


In [ ]:
submission_all_train['Demanda_uni_equil'] = submission_all_train['Demanda_uni_equil'].round(1)
submission_all_train.loc[submission_all_train['Demanda_uni_equil']<0,'Demanda_uni_equil'] = 0
submission_all_train['Demanda_uni_equil'].describe()

In [ ]:
submission_all_train['id'] = submission_all_train['id'].astype(int)

In [36]:
submission_all_train.head()

,id,Demanda_uni_equil
0,1569352,9.9
1,6667200,33.2
2,1592616,18.3
3,3909690,61.5
4,3659672,34.3


In [ ]:
submission_all_train.to('')

### validation 45678 to 10
------------------------

In [45]:
train_pivot_45678_to_10 = pd.read_pickle('validation_45678_10.pickle')
train_pivot_45678_to_10.head()

Semana,id,ruta_freq,clien_freq,agen_freq,prod_freq,agen_for_log_de,ruta_for_log_de,cliente_for_log_de,producto_for_log_de,agen_ruta_for_log_de,...,LR_prod_corr,t_m_6_cum,t_m_5_cum,t_m_4_cum,t_m_3_cum,t_m_2_cum,NombreCliente,weight,weight_per_piece,pieces
0,1569352,10.0,158.0,3861.0,169.0,3.951774,3.165635,2.83547,2.493198,3.165635,...,-1.349546,2.397895,2.397895,2.397895,2.397895,2.397895,131.0,691.0,NaN,NaN
1,6667200,658.0,158.0,12429.0,10746.0,3.509364,3.211487,2.83547,4.374220,2.805172,...,19.839575,3.761200,7.545390,11.041898,14.475884,18.164764,6027.0,740.0,NaN,NaN
2,1592616,658.0,158.0,12429.0,10736.0,3.509364,3.211487,2.83547,3.987994,2.805172,...,17.888412,0.000000,0.000000,2.833213,6.165418,8.730368,6027.0,480.0,NaN,NaN
3,3909690,658.0,158.0,12429.0,13169.0,3.509364,3.211487,2.83547,4.535940,2.805172,...,20.221937,4.158883,8.589700,12.303272,16.620760,20.715105,6027.0,680.0,NaN,NaN
4,3659672,658.0,158.0,12429.0,10624.0,3.509364,3.211487,2.83547,3.268587,2.805172,...,13.291507,1.945910,5.529429,9.084777,12.773657,16.602299,6027.0,567.0,NaN,NaN


In [46]:
dtest_valid = xgb.DMatrix(train_pivot_45678_to_10.drop(['id'],axis =1), missing=np.nan)

In [134]:
submission_10_valid = pd.DataFrame()
submission_10_valid = train_pivot_45678_to_10[['id']].copy()
submission_10_valid['predict'] = bst.predict(dtest_valid)
submission_10_valid.reset_index(drop = True,inplace = True)

In [100]:
submission_10_valid.head()

Semana,id,predict
0,1569352,2.259041
1,6667200,3.567400
2,1592616,2.877977
3,3909690,4.066172
4,3659672,3.513400


In [135]:
submission_all_train = pd.DataFrame()
submission_all_train = pd.concat([submission_10_valid,submission_11_all_train],axis =0)
submission_all_train['predict'] = submission_all_train['predict'].apply(np.expm1)
submission_all_train.rename(columns = {'predict':'Demanda_uni_equil'},inplace = True)
submission_all_train.head()

Semana,id,Demanda_uni_equil
0,1569352,8.406385
1,6667200,34.465729
2,1592616,16.756153
3,3909690,57.540695
4,3659672,33.211864


In [136]:
submission_all_train.loc[submission_all_train['Demanda_uni_equil']<0,'Demanda_uni_equil'] = 0
submission_all_train['Demanda_uni_equil'].describe()

count    6.999251e+06
mean     6.043300e+00
std      1.571901e+01
min      0.000000e+00
25%      1.877748e+00
50%      3.051292e+00
75%      5.550261e+00
max      5.625943e+03
Name: Demanda_uni_equil, dtype: float64

In [137]:
submission_all_train['id'] = submission_all_train['id'].astype(int)

In [138]:
submission_all_train['Demanda_uni_equil'].describe()

count    6.999251e+06
mean     6.043300e+00
std      1.571901e+01
min      0.000000e+00
25%      1.877748e+00
50%      3.051292e+00
75%      5.550261e+00
max      5.625943e+03
Name: Demanda_uni_equil, dtype: float64

In [139]:
submission_all_train.head()

Semana,id,Demanda_uni_equil
0,1569352,8.406385
1,6667200,34.465729
2,1592616,16.756153
3,3909690,57.540695
4,3659672,33.211864


In [123]:
submission_all_train['Demanda_uni_equil'] = submission_all_train['Demanda_uni_equil'].round(1)

In [140]:
submission_all_train.to_csv('valid10_6.csv',index = False)